# **Tutoriel** - Fine-tuning de romans du 17e siècle Falcon-7B-instruct-4bit

<img src="https://raw.githubusercontent.com/opinionscience/FabriqueLLM/main/illustration/falcon_image.png" alt="Falcon logo"  width="500"/>

Ce carnet de code mis à disposition par [OpSci](https://www.opsci.ai/fr/) permet d'effectuer le fine-tuning d'un grand modèle de langue, Falcon-7B avec la **version gratuite de Google Colab** sur un corpus complexe : 6000 extraits de romans en français 17e siècle. C'est un bon cas d'usage où le fine-tuning permet d'obtenir rapidement des résultats plus intéressants que GPT-4.

Créé par le Technology Innovation Institute d'Abu Dhabi, Falcon est aujourd'hui le LLM open source de référence. La principale alternative, Llama, est réservée aux usages de recherche non commerciale. Falcon est disponible sous deux versions : la principale à 40 milliards de paramètres et une version plus légère que nous allons utiliser ici à 7 milliards de paramètres. Ce modèle inclut un corpus plus multilingue que d'autres LLMs ouverts comme Pythia ou MPT.

Ce carnet de code utilise une version déjà ré-entraînée de Falcon-7 : instruct-4-bit. C'est aussi une version plus compacte de Falcon qui devrait pouvoir tourner sur une version gratuite de Google Colab : vous aurez besoin d'environ 10go de Vram. Ce fine-tuning sera aussi plus superficiel mais c'est déjà très pratique pour effectuer de premiers tests.

Cette démonstration ne fait tourner qu'une seule *epoch* ce qui est suffisant pour avoir un premier aperçu. Pour obtenir un bon modèle, il est conseillé de faire tourner le fine-tuning pendant trois *epochs*. Sur notre corpus de démonstration de 6000 instructions une *epoch* prendra un peu plus d'une heure avec les GPUs gratuits de Google Colab. Un fine-tuning complet (trois *epochs*) prendra environ 3h : sur la version gratuite de Google Colab ce type de traitement d'une durée un peu longue risque d'être interrompu.

# Installation

Toute cette section ne doit être exécutée qu'une seule fois afin d'initialiser votre environnement d'installation sur Google Drive. Pour toutes les exécutions suivantes vous pouvez normalement la sauter.

En tout premier lieu nous vérifions si nous disposons de suffisamment de mémoire vive (au moins 24go) sinon ce n'est pas la peine de lancer le script.

In [ ]:
!nvidia-smi

Sun Jun 11 14:57:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    40W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

D'abord nous allons nous maintenant connecter à Google Drive. C'est vraiment recommandé et tout l'intérêt d'utiliser Google Colab. Autrement à l'expiration de la session tout le modèle sera perdu. À noter qu'il y a une latence plus ou moins importante entre Google Colab et Google Drive : vous ne verrez pas immédiatement les fichiers intermédiaires (*checkpoint*) et les fichiers finaux sur Drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd "/content/drive/My Drive/falcon"

Mounted at /content/drive
/content/drive/My Drive/falcon


Nous installons falcontune. C'est une petite application python disponible de Github qui permet d'effectuer le fine-tuning de Falcon.

In [ ]:
!git clone https://github.com/rmihaylov/falcontune.git

Cloning into 'falcontune'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 124 (delta 43), reused 53 (delta 30), pack-reused 49
Receiving objects: 100% (124/124), 63.60 KiB | 4.89 MiB/s, done.
Resolving deltas: 100% (57/57), done.


Et nous récupérons aussi les poids du modèle :

In [ ]:
!wget https://huggingface.co/TheBloke/falcon-7b-instruct-GPTQ/resolve/main/gptq_model-4bit-64g.safetensors

--2023-06-20 16:01:56--  https://huggingface.co/TheBloke/falcon-7b-instruct-GPTQ/resolve/main/gptq_model-4bit-64g.safetensors
Resolving huggingface.co (huggingface.co)... 65.9.86.43, 65.9.86.34, 65.9.86.125, ...
Connecting to huggingface.co (huggingface.co)|65.9.86.43|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/9a/ea/9aea392ba3a1a4fa936207c60a9ba3cfa28fa3b935dfffd3227300a5ff38d088/ceb8ec3d0c432d043ec563d42e2571c94a5e884aedfb5acc6b38612a60490c7b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27gptq_model-4bit-64g.safetensors%3B+filename%3D%22gptq_model-4bit-64g.safetensors%22%3B&Expires=1687536117&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzlhL2VhLzlhZWEzOTJiYTNhMWE0ZmE5MzYyMDdjNjBhOWJhM2NmYTI4ZmEzYjkzNWRmZmZkMzIyNzMwMGE1ZmYzOGQwODgvY2ViOGVjM2QwYzQzMmQwNDNlYzU2M2Q0MmUyNTcxYzk0YTVlODg0YWVkZmI1YWNjNmIzODYxMmE2MDQ5MGM3Yj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0

## Le corpus d'instructions

Nous récupérons notre set d'instruction qui ne sont en réalité pas vraiment des "instructions" mais juste des extraits de romans du 17e siècle (avec la première partie du texte et la suite que le modèle est censé prédire).

In [2]:
!wget https://raw.githubusercontent.com/opinionscience/InstructionFr/main/public_domain/instruct_fr_novel17.json

--2023-06-23 09:50:58--  https://raw.githubusercontent.com/opinionscience/InstructionFr/main/public_domain/instruct_fr_novel17.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8361701 (8.0M) [text/plain]
Saving to: ‘instruct_fr_novel17.json’

instruct_fr_novel17 100%[===================>]   7.97M  --.-KB/s    in 0.1s    

2023-06-23 09:50:59 (66.7 MB/s) - ‘instruct_fr_novel17.json’ saved [8361701/8361701]



Ces instructions utilisent le format classique du projet Alpaca de Stanford : *instructions*, *input* (optionnellement) et *output*. En résumé, les instructions correspondent à des exemples de prompts que pourraient laisser les utilisateurs du LLM, les *outputs* à la réponse que le LLM devrait générer et les *inputs* apporte des éléments de contextes supplémentaires (par exemple sous la forme de textes cités en exemple)

In [3]:
import json

with open('instruct_fr_novel17.json', 'r') as f:
    data = json.load(f)

json_formatted_str = json.dumps(data[0:3], indent=2)

print(json_formatted_str)

[
  {
    "instruction_id": "gbooks_Yvg5AAAAcAAJ.pdf_141",
    "instruction": "El le pafle generalementpour allez grande mais il n'e\u017ft pas aficur\u00e9 que per\u017fonne ne les puifle \u017furpafler & comme tous mes compagnons ont allez bonne opinion de vous pour croire que vous pourriez bien au moins les egalerils m'ont charg\u00e9 de vous \u017folliciter \u00e0 venir di fputer avec nous queiques -uns des prix que la bont\u00e9 du Roi & les aplauditlements du peuple nous ont accordez. Ulfle qui pour plus d'une rai\u017fon n'avoit point envie de \u017fe m\u00ealer dans ces jeux remercia Laodamas le plus honn\u00ea . tement qu'il putlouant beaucoup au re\u017fte& la force & l'adrefle de ceux qui avoient di\u017fput\u00e9 & lur tout de ceux qui s'\u00e9toient dittingu\u00e9s en remportant les prix .",
    "output": "Si Lao damas fut content de cette mode\u017fte repon\u017feil n'en fut pas de m\u00eame de quelques -uns des autres qui \u017fe per\u017fuadant que s'il \u017fe defen do

Nous procédons à l'installation de falcontune. Cela prendra 1-2 minutes. Vous devrez le refaire à chaque nouvelle session même si vous avez déjà chargé l'application sur Google Drive.

In [ ]:
# Installation:
!cd falcontune && pip install -r requirements.txt
!cd falcontune && python setup.py install
# !cd falcontune && python setup_cuda.py install  # if cuda, default is triton

Nous allons maintenant réinitiliser notre environnement de travail pour bien intégrer l'installation de falcontune. Tout va crasher mais c'est normal !

In [ ]:
# Restart:
import os; os.kill(os.getpid(), 9)

# Finetuning du modèle

Tout est prêt à lancer le fine-tuning du modèle. Nous allons juste désactiver Wandb (une extension utilisée par falcontune qui ne présente pas d'intérêt pour nous)

In [1]:
# Disable wandb:
import os; os.environ["WANDB_DISABLED"] = "true"
%cd "/content/drive/My Drive/falcon"

/content/drive/My Drive/falcon


Et nous sommes prêt à lancer la grande commande. Il y a beaucoup de paramètre mais seulement quelqu'uns sont importants :
* Nous allons utiliser le modèle Falcon-7b de base et leurs poids correspondants (tiiuae/falcon-7b)
* Le fine-tuning sera effectué sur le set d'instruction *instruct_fr_novel17.json* (évidemment à changer si vous optez pour un autre jeu de données).
* Les fichiers du modèle seront placés dans le dossier *falcon-7b-novel17c-4bit* (de nouveau à changer pour le nom de votre modèle).
* Nous ne ferons tourner le fine-tuning que sur une *epoch* ce qui est suffisant pour un premier test.
* Nous utilisons un taux d'apprentissage plus élevé que la recommandation par défaut comme le corpus est assez inhabituel.

Après avoir lancé le script, Google Colab va tourner pendant un peu moins de 40 minutes.

Si tout se passe bien vous verrez défiler le processus d'entraînement avec trois indicateurs régulièrement réactualisés : "{'loss': 1.8581, 'learning_rate': 0.0002993736951983298, 'epoch': 0.0}" :
* Le "loss" c'est en quelque sorte le taux d'erreur du modèle : plus cette mesure est basse et plus le modèle parvient à prédire des textes assez approchants de ceux qui sont présent dans le corpus d'instruction.
* Le *learning rate* (taux d'apprentissage) c'est la capacité du modèle à mémoriser de nouveaux éléments mais aussi à en oublier des anciens. Cet indicateur va constamment baisser au fur et à mesure de l'apprentissage.
* L'*epoch* c'est le cycle d'apprentissage. Comme nous n'avons défini qu'une *epoch* cela correspondra à des pourcentages (de 0 à 0.99 à la fin de l'entraînement).

In [2]:
!falcontune finetune \
    --model=falcon-7b-instruct-4bit \
    --weights=gptq_model-4bit-64g.safetensors \
    --dataset=./instruct_fr_novel17.json \
    --data_type=alpaca \
    --lora_out_dir=./falcon-7b-novel17c-4bit/ \
    --mbatch_size=1 \
    --batch_size=2 \
    --epochs=1 \
    --lr=5e-4 \
    --cutoff_len=256 \
    --lora_r=8 \
    --lora_alpha=16 \
    --lora_dropout=0.05 \
    --warmup_steps=5 \
    --save_steps=50 \
    --save_total_limit=3 \
    --logging_steps=5 \
    --target_modules='["query_key_value"]'

2023-06-23 09:53:13.415168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/

Si tout se passe bien, vous devrez avoir comlètement fini l'entraînement. Un petit message final apparaît vous invitant à partager le modèle sur HuggingFace: "Training completed. Do not forget to share your model on huggingface.co/models =)"

Après un petit temps de synchronisation entre Google Colab, vous allez voir apparaître deux fichiers dans le dossier du modèle de fine-tuning : adapter_model.bin (le modèle proprepement dit) et adapter_model.config (un fichier de configuration). À noter que le modèle de fine-tuning est considérablement plus petit que le modèle d'origine : c'est en quelque sorte un modèle complémentaire qui vient ajuster le LLM (et il en aura toujours besoin pour fonctionner).

# Générer du texte

Et maintenant il est possible de générer du texte. La fonction par défaut de falcontune n'est pas pour l'instant pas très pratique mais cela devrait s'améliorer prochainement. À noter aussi que les instructions trop brèves peut susciter un bug un peu agaçant (probability distribution error)

In [11]:
!falcontune generate \
    --model falcon-7b-instruct-4bit \
    --weights gptq_model-4bit-64g.safetensors \
    --lora_apply_dir ./falcon-7b-novel17c-4bit/ \
    --max_new_tokens 200 \
    --use_cache \
    --do_sample \
    --instruction "Pourrois-tu me iustifier le moyen le plus ſimple & le plus aisé que j'ay par lequel je pourrois me rendre sur la Lune ? ###OUTPUT"

2023-06-23 10:27:51.589827: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/